In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyproj import Geod
from tqdm.notebook import tqdm
from matplotlib.animation import FuncAnimation
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import graph_tool.all as gt
import networkx as nx
import pickle

In [3]:
df = pd.read_csv('./VF_data/pandemos_tra_tapas_modell.csv')

In [3]:
df.head(15)

,p_id,taz_id_start,loc_id_start,lon_start,lat_start,taz_id_end,loc_id_end,lon_end,lat_end,start_time_min,travel_time_sec,mode,activity,activity_start_min,activity_duration_min
0,101357880,468,302652,13.159688,52.549478,467,-100792825,13.204950,52.560776,868,1316,2,10,890,145
1,101357880,467,-100792825,13.204950,52.560776,466,302659,13.199601,52.565567,1035,924,0,740,1050,225
2,101357880,466,302659,13.199601,52.565567,467,-100792825,13.204950,52.560776,1275,924,0,10,1290,120
3,100020055,87,-100020055,13.397226,52.536221,728,187669,13.406449,52.463404,329,3655,5,212,390,900
4,100020055,728,187669,13.406449,52.463404,87,-100020055,13.397226,52.536221,1290,3726,5,10,1352,98
5,100868213,203,-100507758,13.421995,52.540396,1108,129362,13.480839,52.524302,487,1890,1,212,519,490
6,100868213,1108,129362,13.480839,52.524302,203,-100507758,13.421995,52.540396,1009,1871,1,10,1040,60
7,100868213,203,-100507758,13.421995,52.540396,199,300023,13.416197,52.546113,1100,1106,0,799,1119,1
8,100868213,199,300023,13.416197,52.546113,226,300285,13.408198,52.540315,1120,978,0,799,1136,65
9,100868213,226,300285,13.408198,52.540315,201,285683,13.414237,52.550004,1201,1208,0,51,1221,45


In [4]:
df.shape

(13578537, 15)

In [5]:
df = df.sort_values(['p_id', 'start_time_min'], ignore_index=True)

In [6]:
df.iloc[0].p_id == df.iloc[1].p_id

True

In [7]:
def get_nnodes(df, n):
    # Returns a list of n lists where each list contains all indices in the df of a certain node
    # The n selected nodes are choosen in order of appearence in df
    # Expects the df to be sorted by p_id and start_time_min

    unique_nodes = df.p_id.unique()
    nnodes = []
    
    if not n:
        n = len(unique_nnodes) # select all nodes if n is not provided

    elif len(unique_nodes) < n: # select at maximum all available nodes in df
        n = len(unique_nodes)

    print(n)

    last_ind, first_ind = 0, 0
    for node in tqdm(unique_nodes[:n], total=n):
        while True:
            if df.iloc[first_ind].p_id != node:
                nnodes.append(list(range(last_ind, first_ind)))
                break

            first_ind += 1
        
        last_ind = first_ind
        
    
    return nnodes, unique_nodes

N = 1000
index2, unique_nnodes = get_nnodes(df, N)

1000


  0%|          | 0/1000 [00:00<?, ?it/s]

In [8]:
def get_position_vector_nnodes(df, n):
    index, unique_nnodes = get_nnodes(df, n)
    geod = Geod("+ellps=WGS84")

    # get edges of time interval
    index_flat = [item for sublist in index for item in sublist]
    tmin = df.iloc[index_flat].start_time_min.min()

    if tmin < 0:
        tmin = abs(tmin)
    else:
        tmin = 0
    tmax = (df.iloc[index_flat].activity_start_min + df.iloc[index_flat].activity_duration_min).max() + tmin

    R_lons, R_lats = np.empty(tmax), np.empty(tmax)

    for i, (ind, id) in tqdm(enumerate(zip(index, unique_nnodes)), total=n):
        # get all df entries of current node
        dfj = df.iloc[ind].sort_values('start_time_min')

        # convert trajectories and activites into position vector R(t)=((lon_0, lat_1), (lon_1, lat_1), ..., (lon_max, lat_max))^T
        S_loni, S_lati = dfj.lon_start.to_numpy(), dfj.lat_start.to_numpy()
        F_loni, F_lati = dfj.lon_end.to_numpy(), dfj.lat_end.to_numpy()
        T = (dfj.travel_time_sec / 60).round(0).astype(int).to_numpy()
        At = dfj.activity_duration_min.to_numpy()
        wait = dfj.start_time_min.iloc[0] + tmin

        R = [(np.nan, np.nan),]*wait
        for lon1, lat1, lon2, lat2, npts, Ati in zip(S_loni, S_lati, F_loni, F_lati, T, At):
            Gi = geod.npts(lon1, lat1, lon2, lat2, npts, initial_idx=0, terminus_idx=0)
            Ai = [(lon2, lat2),]*Ati
            R += Gi + Ai

        R += [(np.nan, np.nan),]*(tmax - len(R))
        R = np.array(R).T
        
        try:
            R_lons = np.vstack((R_lons, R[0]))
            R_lats = np.vstack((R_lats, R[1]))
        except ValueError:
            over_shoot = len(R[0]) - tmax
            print(f'Over shoot due to precission error: {over_shoot}min')
            R_lons = np.vstack((R_lons, R[0, over_shoot:]))
            R_lats = np.vstack((R_lats, R[1, over_shoot:]))

    # Return in format R_x(t)=[[r1(t=0), r2(t=0), ..., rn(t=0)], [r1(t=1), r2(t=1), ..., rn(t=tmax)], ..., [r1(t=tmax), r2(t=tmax), ..., rn(t=tmax)]]
    return R_lons.T, R_lats.T
        
R_lons, R_lats = get_position_vector_nnodes(df, 1000)

1000


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
# %lprun -f get_position_vector_nnodes get_position_vector_nnodes(df, 10_000)

In [10]:
def get_position_vector_nnodes2(df, n):
    index, unique_nnodes = get_nnodes(df, n)
    geod = Geod("+ellps=WGS84")

    # get edges of time interval
    index_flat = [item for sublist in index for item in sublist]
    tmin = df.iloc[index_flat].start_time_min.min()

    if tmin < 0:
        tmin = abs(tmin)
    else:
        tmin = 0
    tmax = (df.iloc[index_flat].activity_start_min + df.iloc[index_flat].activity_duration_min).max() + tmin

    R_lons2, R_lats2 = [], []
    R_lons, R_lats = [], []

    for i, (ind, id) in tqdm(enumerate(zip(index, unique_nnodes)), total=n):
        # get all df entries of current node
        dfj = df.iloc[ind] #.sort_values('start_time_min')

        # convert trajectories and activites into position vector R(t)=((lon_0, lat_1), (lon_1, lat_1), ..., (lon_max, lat_max))^T
        S_loni, S_lati = dfj.lon_start.to_numpy(), dfj.lat_start.to_numpy()
        F_loni, F_lati = dfj.lon_end.to_numpy(), dfj.lat_end.to_numpy()
        T = (dfj.travel_time_sec / 60).round(0).to_numpy(int)
        At = dfj.activity_duration_min.to_numpy()
        wait = dfj.start_time_min.iloc[0] + tmin

        R = [(np.nan, np.nan),]*wait
        for lon1, lat1, lon2, lat2, npts, Ati in zip(S_loni, S_lati, F_loni, F_lati, T, At):
            Gi = geod.npts(lon1, lat1, lon2, lat2, npts, initial_idx=0, terminus_idx=0)
            Ai = [(lon2, lat2),]*Ati
            R += Gi + Ai

        R += [(np.nan, np.nan),]*(tmax - len(R))
        R = list(map(list, zip(*R))) # transpose
        
        R_lons.append(R[0])
        R_lats.append(R[1])
            
    try:
        R_lons = np.array(R_lons)
        R_lats = np.array(R_lats)
    except ValueError:
        print('Over shoot due to precission error')
        length = len(R_lons[0])
        for i, R in enumerate(R_lons):
            if len(R) != length:
                over_shoot = len(R) - length
                R_lons[i] = R_lons[i][over_shoot: ]
                R_lats[i] = R_lats[i][over_shoot: ]
                
                R_lons = np.array(R_lons)
                R_lats = np.array(R_lats)
                break



    # Return in format R_x(t)=[[r1(t=0), r2(t=0), ..., rn(t=0)], [r1(t=1), r2(t=1), ..., rn(t=tmax)], ..., [r1(t=tmax), r2(t=tmax), ..., rn(t=tmax)]]
    return R_lons.T, R_lats.T

# R_lons2, R_lats2 = get_position_vector_nnodes2(df, 5000)

In [11]:
# R_lons2, R_lats2 = get_position_vector_nnodes2(df, 50_000)

In [12]:
# %lprun -f get_position_vector_nnodes2 get_position_vector_nnodes2(df, 50_000)

In [13]:
def setup_plot():
    fname = './shape_files/Berlin/bezirksgrenzen.shp'

    adm1_shapes = list(shpreader.Reader(fname).geometries())

    fig, ax = plt.subplots(figsize=(20, 15), subplot_kw={'projection': ccrs.PlateCarree()})

    ax.add_geometries(adm1_shapes, ccrs.PlateCarree(),
                    edgecolor='black', facecolor='white', alpha=.3)

    ax.set_extent([13.05, 13.78, 52.3, 52.7], ccrs.PlateCarree())

    return fig, ax

In [14]:
# X, Y = get_position_vector_nnodes2(df, 500_000)

In [15]:
# fig, ax = setup_plot()
# scat = ax.scatter(X[0], Y[0], s=1, alpha=.3, transform=ccrs.PlateCarree())
# print(len(X))
d, h, m = 0, 0, 0

def animation_function(i):
    d = int(np.floor(i/1440)) # 60*24=1440 minutes per day
    h = int(np.floor(i/60) - 24*d)
    m = int(i - 60*h - 1440*d)

    print(i)
    data = np.stack((X[i], Y[i])).T
    scat.set_offsets(data)
    ax.set_title(f'{str(d).zfill(2)}:{str(h).zfill(2)}:{str(m).zfill(2)}')

# anim = FuncAnimation(fig, animation_function, frames=len(X), interval=1)
# anim.save('berlin.gif', fps=50)

##### Wo befinden sich die Locations mit den meisten Besuchern?
##### Wo befinden sich die einzelnen Haushalte?

In [16]:
def find_loc_most_visits(df, first_n= 1000):
    # Return first n locations with the most visits
    # A visit is counted, if its the end of a trajectory
    dfg = df.groupby('loc_id_end')
    dfg.size()
    df_locations = dfg[['lon_end', 'lat_end']].first()
    df_locations['visits'] = dfg.size()
    loc_lon_lat_size = df_locations.sort_values('visits', ascending=False)[:first_n].to_numpy()
    print(loc_lon_lat_size[:4])

    fig, ax = setup_plot()
    ax.set_title(f'{first_n}-most visited location', fontsize=20)
    sc = ax.scatter(loc_lon_lat_size[:, 0], loc_lon_lat_size[:, 1], loc_lon_lat_size[:, 2]/100, c='blue', alpha=.7, edgecolors='black')
    plt.legend(*sc.legend_elements("sizes", num=4), title='visits in units of 100', title_fontsize=15)
    plt.tight_layout()
    plt.savefig(f'locations_berlin_{first_n}.png')
    plt.close()

find_loc_most_visits(df, 10_000)

[[1.33291608e+01 5.25149479e+01 7.86470000e+04]
 [1.33054974e+01 5.25096848e+01 7.14140000e+04]
 [1.33346602e+01 5.25935055e+01 5.92370000e+04]
 [1.35569618e+01 5.25600620e+01 4.74050000e+04]]


In [17]:
def find_households(first_n=None):
    household_lon_lat = df.groupby('p_id')[['lon_end', 'lat_end']].last().to_numpy()
    fig, ax = setup_plot()

    ax.set_title('Berlin household distribution', fontsize=20)
    ax.scatter(household_lon_lat[:first_n, 0], household_lon_lat[:first_n, 1], s=1, alpha=.5)

    plt.tight_layout()
    plt.savefig(f'berlin_household_distribution_first{first_n}.png')
    plt.close()

find_households()

##### Effizientes generieren der Netzwerke

In [23]:
df['activity_end_min'] = df['activity_start_min'] + df['activity_duration_min']
df.activity_end_min

0            651
1            897
2            916
3           1224
4           1215
            ... 
13578532    1505
13578533    1231
13578534    1293
13578535    1105
13578536    1172
Name: activity_end_min, Length: 13578537, dtype: int64

In [27]:
contacts = []

def get_contacts(grp):
    print(grp)
    print(grp['activity_end_min'].apply(lambda end: grp['activity_start_min'] < end))
    print('\n\n\n')
    

df.groupby('loc_id_end')[['p_id', 'activity_start_min', 'activity_end_min']].apply(get_contacts)

               p_id  activity_start_min  activity_end_min
13578525  103604315                1000              1193
13578527  103604316                 972               982
13578529  103604316                1043              1298
13578532  103604316                1503              1505
13578534  103604317                1253              1293
13578536  103604318                1124              1172
          13578525  13578527  13578529  13578532  13578534  13578536
13578525      True      True      True     False     False      True
13578527     False      True     False     False     False     False
13578529      True      True      True     False      True      True
13578532      True      True      True      True      True      True
13578534      True      True      True     False      True      True
13578536      True      True      True     False     False      True




               p_id  activity_start_min  activity_end_min
13578506  103604310                1105          

In [47]:
with open(f'./pickles/simple_edges_{0}.pkl', 'rb') as f:
    edge_dict = pickle.load(f)

sub_dict = {}
for key in [103604317, 103604316, 103604315]:
    sub_dict[key - 103604300] = -edge_dict[103604317]+  103604300

sub_dict

{17: array([3436022,  242019,  156320,  150933,  114335,   95383,   90797,
          78285,   63337,   62956,   48368,   48265,   47572,   34477,
          29541,    7820,     -17]),
 16: array([3436022,  242019,  156320,  150933,  114335,   95383,   90797,
          78285,   63337,   62956,   48368,   48265,   47572,   34477,
          29541,    7820,     -17]),
 15: array([3436022,  242019,  156320,  150933,  114335,   95383,   90797,
          78285,   63337,   62956,   48368,   48265,   47572,   34477,
          29541,    7820,     -17])}

In [35]:
G_nx = nx.Graph(sub_dict)

In [62]:
G_gt = gt.Graph()
G_gt.add_edge_list([(10, 1)])

In [12]:
# Your sample DataFrame
df = pd.DataFrame({'p_id': [0, 0, 0, 1, 2, 2, 3],
                   'start': [0, 100, 200, 50, 20, 40, 150],
                   'end': [50, 110, 205, 80, 25, 50, 170]})

# Define a function to generate the required array
def generate_array(row):
    return np.arange(row['start'], row['end'] + 1)

def get_position(grp):
    indices = np.concatenate(grp.to_numpy())
    print(grp.name)
    print(indices)

# Apply the function to each row in the DataFrame
result = df.apply(generate_array, axis=1)

# Group the arrays by 'p_id' and convert them into NumPy arrays
result.groupby(df['p_id']).apply(get_position)#.reset_index(name='result')

result

0
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50 100 101 102
 103 104 105 106 107 108 109 110 200 201 202 203 204 205]
1
[50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
 74 75 76 77 78 79 80]
2
[20 21 22 23 24 25 40 41 42 43 44 45 46 47 48 49 50]
3
[150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167
 168 169 170]


0    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
1    [100, 101, 102, 103, 104, 105, 106, 107, 108, ...
2                       [200, 201, 202, 203, 204, 205]
3    [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 6...
4                             [20, 21, 22, 23, 24, 25]
5         [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
6    [150, 151, 152, 153, 154, 155, 156, 157, 158, ...
dtype: object